In [25]:
# cell 1: Import required libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import psutil
import gc

print("All libraries imported successfully!")

All libraries imported successfully!


In [39]:
# cell 2: Memory monitoring utilities to prevent RAM crash
def monitor_memory():
    """Check current memory usage"""
    process = psutil.Process()
    memory_usage = process.memory_info().rss / 1024 / 1024  # Convert to MB
    print(f"Current memory usage: {memory_usage:.2f} MB")
    return memory_usage

def clear_memory():
    """Clear memory and TensorFlow session"""
    gc.collect()
    keras.backend.clear_session()
    print("Memory cleared successfully!")

# Check initial memory
monitor_memory()

Current memory usage: 342.50 MB


342.5

In [40]:
# cell 3: Custom Data Generator for batch loading - prevents RAM overload
class EyeDataGenerator(keras.utils.Sequence):
    """
    Custom generator to load images in batches
    Handles 80K+ images without RAM crash
    """
    def __init__(self, image_paths, labels, batch_size=32, img_size=(64, 64), shuffle=True):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.on_epoch_end()
    
    def __len__(self):
        """Return number of batches per epoch"""
        return int(np.ceil(len(self.image_paths) / self.batch_size))
    
    def __getitem__(self, index):
        """Get one batch of data"""
        # Get batch paths and labels
        batch_paths = self.image_paths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_labels = self.labels[index * self.batch_size:(index + 1) * self.batch_size]
        
        # Load images one by one
        batch_images = []
        for img_path in batch_paths:
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, self.img_size)
                img = img.astype('float32') / 255.0  # Normalize
                batch_images.append(img)
            else:
                # If image fails to load, add blank image
                batch_images.append(np.zeros(self.img_size, dtype='float32'))
        
        # Handle cases where some images failed to load
        actual_batch_size = len(batch_images)
        if actual_batch_size < len(batch_labels):
            batch_labels = batch_labels[:actual_batch_size]
        
        # Reshape for CNN (add channel dimension)
        batch_images = np.array(batch_images).reshape(-1, self.img_size[0], self.img_size[1], 1)
        
        return batch_images, np.array(batch_labels)
    
    def on_epoch_end(self):
        """Shuffle data after each epoch"""
        if self.shuffle:
            indices = np.arange(len(self.image_paths))
            np.random.shuffle(indices)
            self.image_paths = self.image_paths[indices]
            self.labels = self.labels[indices]

print("Data Generator class defined successfully!")

Data Generator class defined successfully!


In [41]:
# cell 4: Collect paths for OPEN EYES images only - no image loading yet
dataset_path = "C:\\Users\\A\\Downloads\\archive\\mrleyedataset"
open_eyes_path = os.path.join(dataset_path, "Open-Eyes")

open_eye_paths = []
open_eye_labels = []

print("Collecting OPEN EYES image paths...")

# Get all open eyes image paths
for img_name in os.listdir(open_eyes_path):
    img_path = os.path.join(open_eyes_path, img_name)
    if os.path.isfile(img_path) and img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        open_eye_paths.append(img_path)
        open_eye_labels.append(1)  # 1 for open eyes

print(f"Open eyes images found: {len(open_eye_paths)}")
monitor_memory()

Open eyes images found: 42952
Current memory usage: 357.16 MB


357.16015625

In [42]:
# cell 5: Collect paths for CLOSE EYES images only - no image loading yet
close_eyes_path = os.path.join(dataset_path, "Close-Eyes")

close_eye_paths = []
close_eye_labels = []

print("Collecting CLOSE EYES image paths...")

# Get all close eyes image paths
for img_name in os.listdir(close_eyes_path):
    img_path = os.path.join(close_eyes_path, img_name)
    if os.path.isfile(img_path) and img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        close_eye_paths.append(img_path)
        close_eye_labels.append(0)  # 0 for closed eyes

print(f"Close eyes images found: {len(close_eye_paths)}")
monitor_memory()

Close eyes images found: 41946
Current memory usage: 371.27 MB


371.2734375

In [43]:
# cell 6: Combine both open and close eyes paths
all_image_paths = np.array(open_eye_paths + close_eye_paths)
all_labels = np.array(open_eye_labels + close_eye_labels)

print(f"TOTAL DATASET SIZE: {len(all_image_paths)} images")
print(f"Open eyes: {len(open_eye_paths)}")
print(f"Close eyes: {len(close_eye_paths)}")
print(f"Memory usage after collecting paths:")
monitor_memory()

# Clear individual lists to save memory
del open_eye_paths, open_eye_labels, close_eye_paths, close_eye_labels
clear_memory()

TOTAL DATASET SIZE: 84898 images
Open eyes: 42952
Close eyes: 41946
Memory usage after collecting paths:
Current memory usage: 372.59 MB
Memory cleared successfully!


In [44]:
# cell 7: Split image paths (not images) into training and testing sets
train_paths, test_paths, train_labels, test_labels = train_test_split(
    all_image_paths, 
    all_labels, 
    test_size=0.2, 
    random_state=42, 
    stratify=all_labels  # Maintain class balance
)

print("Data split completed:")
print(f"Training samples: {len(train_paths)}")
print(f"Testing samples: {len(test_paths)}")
print(f"Training open eyes: {np.sum(train_labels == 1)}")
print(f"Training close eyes: {np.sum(train_labels == 0)}")

monitor_memory()

Data split completed:
Training samples: 67918
Testing samples: 16980
Training open eyes: 34361
Training close eyes: 33557
Current memory usage: 360.78 MB


360.78125

In [45]:
# cell 8: Create data generators for training and testing
batch_size = 32  # Adjust based on your RAM
img_size = (64, 64)

train_generator = EyeDataGenerator(train_paths, train_labels, batch_size, img_size)
test_generator = EyeDataGenerator(test_paths, test_labels, batch_size, img_size, shuffle=False)

print("Data generators created successfully!")
print(f"Training batches per epoch: {len(train_generator)}")
print(f"Testing batches: {len(test_generator)}")
print(f"Each batch contains {batch_size} images")

monitor_memory()

Data generators created successfully!
Training batches per epoch: 2123
Testing batches: 531
Each batch contains 32 images
Current memory usage: 382.04 MB


382.03515625

In [55]:
# Cell 9: Create Optimized CNN Model (BEST for Eye Detection)
def create_optimized_eye_model():
    """
    OPTIMIZED Model for Eye Detection:
    - More layers for better feature extraction
    - Better dropout to prevent overfitting  
    - Fine-tuned for 80K+ dataset
    - NO AUGMENTATION needed
    """
    model = keras.Sequential([
        # Conv Block 1 - Basic Features
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Conv Block 2 - Complex Features  
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.3),
        
        # Conv Block 3 - Advanced Features
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.35),
        
        # Conv Block 4 - High-level Features (NEW)
        layers.Conv2D(256, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.5),
        
        # Dense Layers for Classification
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')  # Open/Close
    ])
    
    # Compile with optimal settings
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )
    
    return model

# Create the optimized model
print("Creating OPTIMIZED Eye Detection Model...")
model = create_optimized_eye_model()
print("✅ Optimized Model Created Successfully!")
model.summary()

Creating OPTIMIZED Eye Detection Model...
✅ Optimized Model Created Successfully!


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)                   │ (None, 62, 62, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 62, 62, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 60, 60, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 30, 30, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 30, 30, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_12 (Conv2D)                   │ (None, 28, 28, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 28, 28, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 26, 26, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 13, 13, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 13, 13, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 11, 11, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 11, 11, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_15 (Conv2D)                   │ (None, 9, 9, 128)           │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_16 (Conv2D)                   │ (None, 2, 2, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 2, 2, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_2           │ (None, 256)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 256)                 │              

 Total params: 682,337 (2.60 MB)

 Trainable params: 681,377 (2.60 MB)

 Non-trainable params: 960 (3.75 KB)

In [ ]:
# Cell 10: Setup Advanced Training Callbacks
print("Setting up ENHANCED training callbacks...")

callbacks = [
    # Early Stopping - Prevent overfitting
    keras.callbacks.EarlyStopping(
        monitor='val_accuracy',  # Stop when accuracy stops improving
        patience=7,              # Increased patience
        restore_best_weights=True,
        verbose=1,
        mode='max'
    ),
    
    # Reduce Learning Rate - Better convergence
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,              # Reduce LR by half
        patience=4,              # Wait for 4 epochs
        min_lr=1e-7,             # Minimum learning rate
        verbose=1
    ),
    
    # Save Best Model - Auto-save best weights
    keras.callbacks.ModelCheckpoint(
        'best_eye_model_v2.h5',
        monitor='val_accuracy',
        save_best_only=True,
        save_weights_only=False,
        verbose=1,
        mode='max'
    ),
    
    # CSV Logger - Track training history
    keras.callbacks.CSVLogger(
        'training_history.csv',
        separator=',',
        append=False
    )
]

print("✅ Enhanced Callbacks Setup Completed!")
print("• Early Stopping (patience=7)")
print("• Learning Rate Scheduler") 
print("• Model Checkpoint")
print("• Training Logger")

# Cell 11: OPTIMIZED Model Training
print("=== STARTING OPTIMIZED MODEL TRAINING ===")
print(" Dataset: 80K+ Eye Images")
print(" Model: Enhanced CNN (4 Conv Blocks)")
print(" This will take time... Please wait!")

monitor_memory()

# Training parameters
initial_epoch = 0
total_epochs = 25

print(f"Training for {total_epochs} epochs...")
print(f"Training samples: {len(train_paths)}")
print(f"Validation samples: {len(test_paths)}")

try:
    # Start OPTIMIZED training
    history = model.fit(
        train_generator,
        epochs=total_epochs,
        initial_epoch=initial_epoch,
        validation_data=test_generator,
        callbacks=callbacks,
        verbose=1
    )
    
    # Training results
    final_train_acc = history.history['accuracy'][-1]
    final_val_acc = history.history['val_accuracy'][-1]
    
    print("\n" + "="*50)
    print("TRAINING COMPLETED SUCCESSFULLY!")
    print("="*50)
    print(f"✅ Final Training Accuracy: {final_train_acc:.4f}")
    print(f"✅ Final Validation Accuracy: {final_val_acc:.4f}")
    print(f"✅ Best Model Saved: 'best_eye_model_v2.h5'")
    print(f"✅ Training Log: 'training_history.csv'")
    
except KeyboardInterrupt:
    print("\n⚠️ Training interrupted by user!")
    
except Exception as e:
    print(f"\n❌ Training Error: {e}")
    print("Trying with reduced epochs...")
    
    # Fallback training
    history = model.fit(
        train_generator,
        epochs=10,
        validation_data=test_generator,
        verbose=1
    )

finally:
    monitor_memory()

In [60]:
# cell 11: Save the final trained model
model.save('eye_detection_model.h5')
print("Final model saved as 'eye_detection_model.h5'")

# Clear training memory
clear_memory()

Final model saved as 'eye_detection_model.h5'
Memory cleared successfully!


In [61]:
# cell 12: Evaluate model on test data
print("=== MODEL EVALUATION ===")

test_loss, test_accuracy, test_precision, test_recall = model.evaluate(
    test_generator,
    verbose=1
)

print("\n=== FINAL RESULTS ===")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}") 
print(f"Test Recall: {test_recall:.4f}")
print(f"Test Loss: {test_loss:.4f}")

=== MODEL EVALUATION ===
531/531 ━━━━━━━━━━━━━━━━━━━━ 1214s 2s/step - accuracy: 0.9904 - loss: 0.0293 - precision: 0.9890 - recall: 0.9918

=== FINAL RESULTS ===
Test Accuracy: 0.9904
Test Precision: 0.9905
Test Recall: 0.9906
Test Loss: 0.0289


In [68]:
# Cell 18: Camera Fixed Sleep Detection System
import time

class SleepDetectionSystem:
    def __init__(self, model_path):
        print("🔄 Loading AI model...")
        self.model = keras.models.load_model(model_path)
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        self.eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
        
        self.eyes_closed_start_time = None
        self.eyes_closed_duration = 0
        self.blink_count = 0
        
    def preprocess_eye(self, eye_roi):
        eye_gray = cv2.cvtColor(eye_roi, cv2.COLOR_BGR2GRAY)
        eye_resized = cv2.resize(eye_gray, (64, 64))
        eye_normalized = eye_resized.astype('float32') / 255.0
        return eye_normalized.reshape(1, 64, 64, 1)
    
    def run_sleep_detection(self):
        """Try different camera indexes"""
        print("📷 Trying to access camera...")
        
        # Try different camera indexes
        for camera_index in [0, 1, 2]:
            print(f"🔄 Trying camera index {camera_index}...")
            cap = cv2.VideoCapture(camera_index)
            
            if cap.isOpened():
                print(f"✅ Camera {camera_index} opened successfully!")
                break
            else:
                print(f"❌ Camera {camera_index} failed")
                cap.release()
        else:
            print("❌ No camera found! Check webcam connection.")
            return
        
        print("🚀 SLEEP DETECTION STARTED!")
        print("👁️ Look at the camera for eye detection")
        print("⏹️ Press 'Q' to quit")
        
        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    print("❌ Failed to grab frame")
                    break
                
                frame = cv2.flip(frame, 1)
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                
                # Display basic frame (even without face detection)
                cv2.putText(frame, "CAMERA WORKING - Looking for face...", (50, 50), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                
                # Try face detection
                faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)
                
                if len(faces) > 0:
                    cv2.putText(frame, "FACE DETECTED!", (50, 80), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                
                current_eyes_closed = 0
                total_eyes_detected = 0
                
                for (x, y, w, h) in faces:
                    # Draw face rectangle
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                    
                    face_roi = frame[y:y+h, x:x+w]
                    face_gray = gray[y:y+h, x:x+w]
                    
                    # Eye detection
                    eyes = self.eye_cascade.detectMultiScale(face_gray, 1.1, 4)
                    
                    for (ex, ey, ew, eh) in eyes:
                        eye_roi = face_roi[ey:ey+eh, ex:ex+ew]
                        
                        # Prediction
                        eye_input = self.preprocess_eye(eye_roi)
                        prediction = self.model.predict(eye_input, verbose=0)[0][0]
                        is_open = prediction > 0.5
                        
                        color = (0, 255, 0) if is_open else (0, 0, 255)
                        cv2.rectangle(frame, (x+ex, y+ey), (x+ex+ew, y+ey+eh), color, 2)
                        
                        status = "OPEN" if is_open else "CLOSED"
                        cv2.putText(frame, status, (x+ex, y+ey-10), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                        
                        if not is_open:
                            current_eyes_closed += 1
                        total_eyes_detected += 1
                
                # Sleep detection logic
                if total_eyes_detected >= 2:
                    if current_eyes_closed >= 2:
                        if self.eyes_closed_start_time is None:
                            self.eyes_closed_start_time = time.time()
                        else:
                            self.eyes_closed_duration = time.time() - self.eyes_closed_start_time
                            
                            if self.eyes_closed_duration > 3.0:
                                alert_level = "🚨 DEEP SLEEP!"
                                alert_color = (0, 0, 255)
                            elif self.eyes_closed_duration > 2.0:
                                alert_level = "⚠️ SLEEP DETECTED!"
                                alert_color = (0, 165, 255)
                            elif self.eyes_closed_duration > 1.0:
                                alert_level = "😴 DROWSY"
                                alert_color = (0, 255, 255)
                            else:
                                alert_level = "👀 Eyes Closed"
                                alert_color = (255, 255, 0)
                                
                            cv2.putText(frame, alert_level, (50, 120), 
                                       cv2.FONT_HERSHEY_SIMPLEX, 1, alert_color, 3)
                    else:
                        if self.eyes_closed_start_time is not None:
                            if time.time() - self.eyes_closed_start_time > 0.3:
                                self.blink_count += 1
                            self.eyes_closed_start_time = None
                            self.eyes_closed_duration = 0
                
                # Dashboard
                cv2.rectangle(frame, (0, 0), (350, 150), (0, 0, 0), -1)
                cv2.putText(frame, f"Blinks: {self.blink_count}", (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(frame, f"Eyes Closed: {self.eyes_closed_duration:.1f}s", (10, 60), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(frame, f"Faces: {len(faces)}", (10, 90), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(frame, "Press Q to quit", (10, 120), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                
                cv2.imshow('⚡ SLEEP DETECTOR ⚡', frame)
                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                    
        finally:
            cap.release()
            cv2.destroyAllWindows()
            print("🛑 Sleep detection stopped")

print("✅ SleepDetectionSystem class defined!")

✅ SleepDetectionSystem class defined!


In [69]:
# Cell 19: Ab yeh run karo
print("🚀 LAUNCHING ELECTRIC SPEED SLEEP DETECTOR...")
print("⚡ Powered by 99.04% Accurate AI Model")

try:
    sleep_detector = SleepDetectionSystem('best_eye_model_v2.h5')
    sleep_detector.run_sleep_detection()
except Exception as e:
    print(f"❌ Error: {e}")

🚀 LAUNCHING ELECTRIC SPEED SLEEP DETECTOR...
⚡ Powered by 99.04% Accurate AI Model
🔄 Loading AI model...


📷 Trying to access camera...
🔄 Trying camera index 0...
✅ Camera 0 opened successfully!
🚀 SLEEP DETECTION STARTED!
👁️ Look at the camera for eye detection
⏹️ Press 'Q' to quit
🛑 Sleep detection stopped


KeyboardInterrupt: 